In [31]:
import sys
sys.path.append('src')
import matplotlib.pyplot as plt
from sts_sensor import StsSensor
import numpy as np
from fitters import *

from boost_histogram import Histogram
from boost_histogram.axis import Regular


In [32]:
data = []
with open('measurements.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if "None" not in line:
            values = line.split(',')
            hits = [[float(x),float(y),float(z)] for x, y, z in zip(values[0::3], values[1::3], values[2::3])]

            data.append(hits)

In [33]:
data = np.array(data)
data.shape

(703, 8, 3)

In [ ]:
matrices = {}
for s in enumerate(sensors):
    matrices[s.idx] = StsSensor(s).get_matrix()

def glb_chi2(align_params):
    theta = align_params[-2]
    phi = align_params[-1]

    Rz = np.array([
        [np.cos(phi), -np.sin(phi), 0],
        [np.sin(phi),  np.cos(phi), 0],
        [0,            0,           1]
    ])

    Ry = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0,             1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])

    R = Rz @ Ry

    T = np.array(align_params[0:3])

    chi2 = 0
    for track in data:
        track = R @ track + T
        r0, tx, ty, residuals = fit_pca(track)
        chi2 += np.sum(residuals**2)
    return chi2

In [40]:
align_params = np.zeros(5)
glb_chi2(align_params)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 8 is different from 3)

In [ ]:
sensors = [StsSensor((0,0,10*i), 6, 0, 0, (0.1, 0.01, 0.0)) for i in range(8)]
h_res_rho = [Histogram(Regular(100, -0.05, +0.05)) for _ in sensors]

for track in data:
    r0, tx, ty, residuals = fit_least_squares(track)
    # print(residuals)
    for sensor_idx, res in enumerate(residuals):
        h_res_rho[sensor_idx].fill(res)

fig, axes = plt.subplots(len(h_res_rho), 1, figsize=(4, 1 * len(h_res_rho)), sharex=True, gridspec_kw={'hspace': 0})
for ax, hist in zip(axes, h_res_rho):
    ax.bar(hist.axes[0].centers, hist.values(), width=hist.axes[0].widths, align='center')
    ax.set_ylabel("")
    ax.grid(True)

axes[-1].set_xlabel("Residual (cm)")
plt.tight_layout()


In [ ]:

# Plotting the sensors edges in 3D space
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot(111, projection='3d')
for sensor in sensors:
    ax.plot([sensor.x - sensor.dx/2, sensor.x + sensor.dx/2, sensor.x + sensor.dx/2, sensor.x - sensor.dx/2, sensor.x - sensor.dx/2],
            [sensor.y - sensor.dy/2, sensor.y - sensor.dy/2, sensor.y + sensor.dy/2, sensor.y + sensor.dy/2, sensor.y - sensor.dy/2],
            [sensor.z, sensor.z, sensor.z, sensor.z, sensor.z], color='blue', alpha=0.5)
ax.set_xlabel('X (cm)')
ax.set_ylabel('Y (cm)')
ax.set_zlabel('Z (cm)')

hits = data[5]

# Plotting 3D space
for hit in hits:
    ax.plot([hit[0]], [hit[1]], [hit[2]], 'bo', label='Hit')
    ax.set_xlabel('X (cm)')
    ax.set_ylabel('Y (cm)')
    ax.set_zlabel('Z (cm)')
    # building the fitted line

r0, tx, ty, residuals = fit_pca(hits)
z = np.linspace(hits[0][2], hits[-1][2], 100)
x = hits[0][0] + tx * (z - hits[0][0])
y = hits[0][1] + ty * (z - hits[0][1])
ax.plot(x, y, z, 'r-', label='Fitted Line')

# Plotting 2D projections
# Plot XZ projection
plt.figure(figsize=(6,3))
plt.subplot(1, 2, 1)
plt.scatter([p[2] for p in hits], [p[0] for p in hits], label='Hits')
plt.plot(z, x, 'r-', label='Fitted Line')
plt.xlabel('Z (cm)')
plt.ylabel('X (cm)')
plt.title('XZ Projection')
plt.legend()

# Plot YZ projection
plt.subplot(1, 2, 2)
plt.scatter([p[2] for p in hits], [p[1] for p in hits], label='Hits')
plt.plot(z, y, 'r-', label='Fitted Line')
plt.xlabel('Z (cm)')
plt.ylabel('Y (cm)')
plt.title('YZ Projection')
plt.legend()